In [1]:
import torch
import numpy as np
from scipy.io import loadmat
import argparse
from pinn.neural_net import MLP
from pinn.util import log
from pathlib import Path
import matplotlib.pyplot as plt
from pinn.evaluate import get_pars, evaluate, load_data, get_residuals, get_loss
import pandas as pd
import glob
import re

c:\Users\marlon\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_ref_case(save_path):

    nt = 101
    nx = 101
    ny = 101

    t = np.linspace(0,1,nt)
    x = np.linspace(0,1,nx)
    y = np.linspace(0,1,ny)

    [x_grid, t_grid, y_grid] = np.meshgrid(x,t,y)
    X = np.hstack((t_grid.flatten()[:,None],x_grid.flatten()[:,None],y_grid.flatten()[:,None]))

    data = loadmat(save_path)

    U_data = data['uref'].transpose((2,1,0))
    V_data = data['vref'].transpose((2,1,0))

    return U_data, V_data

In [3]:
def get_case(case,U_ref,V_ref):
    nt = 101
    nx = 101
    ny = 101

    t = np.linspace(0,1,nt)
    x = np.linspace(0,1,nx)
    y = np.linspace(0,1,ny)


    [U, V, t_grid, x_grid, y_grid, X] = evaluate(t,x,y,case)

    E_rms = np.sqrt(np.sum((U-U_ref)**2)+np.sum((V-V_ref)**2))/np.sqrt(2*nt*nx*ny)
    E_abs = np.sum(np.abs(U-U_ref)+np.abs(V-V_ref))/(2*nt*nx*ny)

    [r1, r2, t_grid, x_grid, y_grid, X] = get_residuals(t,x,y,case)

    R_rms = np.sqrt(np.sum(r1**2) + np.sum(r2**2))/(np.sqrt(nt*nx*ny))
    R_abs = (np.sum(np.abs(r1)) + np.sum(np.abs(r2)))/(nt*nx*ny)

    m = torch.load(case)

    log_loss = m['loss']
    time = m['time']
    memory = m['memory']

    #return {'case':case,'t':t,'x':x,'y':y,'E':E,'U':U,'V':V,'R_rms':R_rms,'R_abs':R_abs,'loss':log_loss,'time':time,'memory':memory}
    return R_rms,R_abs,E_rms,E_abs,time,memory

In [4]:
def summarize(nf,ns,U_ref,V_ref):
    cases = [f for f in glob.glob(f"models/model_nf{nf}_ns{ns}_MLPRes_2x2x20_shuffle0_seed*_adam_lr0.01_loss_l1.pt")]

    n_cases = len(cases)

    R_rms = 0
    R_abs = 0
    E_rms = 0
    E_abs = 0
    time = 0
    memory = 0

    for case in cases:
        R_rmsC,R_absC,E_rmsC,E_absC,timeC,memoryC = get_case(case,U_ref,V_ref)
        R_rms += R_rmsC/n_cases
        R_abs += R_absC/n_cases
        E_rms += E_rmsC/n_cases
        E_abs += E_absC/n_cases
        time += timeC/n_cases
        memory += memoryC/n_cases

    return R_rms,R_abs,E_rms,E_abs,time,memory,n_cases


In [5]:
nfs = [0,100,1000,10000,100000]
nss = [0,100,1000,10000,100000]
n_nf = len(nfs)
n_ns = len(nss)

U_ref,V_ref = get_ref_case('reference_solution.mat')

R_rms = np.zeros((n_nf,n_ns))
R_abs = np.zeros((n_nf,n_ns))
E_rms = np.zeros((n_nf,n_ns))
E_abs = np.zeros((n_nf,n_ns))
time = np.zeros((n_nf,n_ns))
memory = np.zeros((n_nf,n_ns))
n_cases = np.zeros((n_nf,n_ns))

for i in range(0,n_nf):
    for j in range(0,n_ns):
        R_rmsC,R_absC,E_rmsC,E_absC,timeC,memoryC,n_casesC = summarize(nfs[i],nss[j],U_ref,V_ref)

        R_rms[i,j] = R_rmsC
        R_abs[i,j] = R_absC
        E_rms[i,j] = E_rmsC
        E_abs[i,j] = E_absC
        time[i,j] = timeC
        memory[i,j] = memoryC
        n_cases[i,j] = n_casesC

In [7]:
all_vars = {'R_rms':R_rms, 'R_abs':R_abs, 'E_rms':E_rms, 'E_abs':E_abs, 'time':time, 'memory':memory}
var_to_view = 'R_rms'

In [10]:
D = pd.DataFrame(all_vars[var_to_view],
                 columns=[f'ns = {ns}' for ns in nss],
                 index=[f'nf = {nf}' for nf in nfs])

D

,ns = 0,ns = 100,ns = 1000,ns = 10000,ns = 100000
nf = 0,0.0,0.0,0.000000,0.0,0.0
nf = 100,0.0,0.0,0.000000,0.0,0.0
nf = 1000,0.0,0.0,0.000000,0.0,0.0
nf = 10000,0.0,0.0,1.508815,0.0,0.0
nf = 100000,0.0,0.0,0.000000,0.0,0.0
